In [1]:
import os
import clip
import torch
from torchvision.datasets import CIFAR100
import math
from pytorch_lightning.loggers import WandbLogger
from tqdm import tqdm
import numpy as np
import torch
import pytorch_lightning as pl
from filelock import FileLock
from torch.utils.data import DataLoader, random_split
from torch.nn import functional as F
from torchvision.datasets import MNIST
from torchvision import transforms
import os
import torchvision
import clip
from clip_lt.utils.labels_names import labels_names
import torch.nn as nn
from PIL import Image

import numpy as np
from sklearn.linear_model import LogisticRegression
from torch.utils.data import DataLoader
from torchvision.datasets import CIFAR100
from tqdm import tqdm

# Load the model
device = "cuda" if torch.cuda.is_available() else "cpu"
clip_model, preprocess = clip.load('ViT-B/32', device)
clip_model.eval()
pass

In [2]:
dataset_dir_path = '/Volumes/black_ssd/datasets/imagenet_lt/'
# dataset_dir_path = '/Users/rotemisraeli/Documents/datasets/TinyImageNet/'

test_data = torchvision.datasets.ImageFolder(dataset_dir_path+'val/',transform=preprocess)
val_data_loader = DataLoader(test_data, batch_size=1,num_workers=4)
train_data = torchvision.datasets.ImageFolder(dataset_dir_path+'train/',transform=preprocess)
train_data_loader = DataLoader(train_data, batch_size=1,num_workers=4)

text_features = torch.load('../text_features.pt')
text_features = text_features / text_features.norm(dim=-1, keepdim=True)
logit_scale = (nn.Parameter(torch.ones([]) * np.log(1 / 0.07))).exp()

In [3]:
def get_features(dataset):
    all_features = []
    all_labels = []

    with torch.no_grad():
        for images, labels in tqdm(DataLoader(dataset, batch_size=100,shuffle=True)):
            features = clip_model.encode_image(images.to(device))

            all_features.append(features)
            all_labels.append(labels)

    return torch.cat(all_features).cpu().numpy(), torch.cat(all_labels).cpu().numpy()

In [4]:
test_features, test_labels = get_features(test_data)


100%|██████████| 200/200 [1:15:38<00:00, 22.69s/it]


In [ ]:
train_features, train_labels = get_features(train_data)


  3%|▎         | 30/1159 [11:33<7:14:58, 23.12s/it]

In [ ]:
classifier = LogisticRegression(random_state=0, C=0.316, max_iter=1000, verbose=1)
classifier.fit(train_features, train_labels)

In [ ]:
predictions = classifier.predict(test_features)
accuracy = np.mean((test_labels == predictions).astype(np.float)) * 100.
print(f"Accuracy = {accuracy:.3f}")
#40%